# the-stack-yaml-k8s

This is a subset of the stack dataset.
Dataset used for building substratus-k8s-instruct an LLM that takes
a prompt expressing your desired K8s resources and generates valid K8s
YAML file.

Dataset contains 2 fields:
| Field       | Description |
| ----------- | ----------- |
| Prompt      | The natural language needed to generate a K8s YAML file |
| YAML        | A valid K8s YAML file that would account |

In [18]:
! pip install datasets kubernetes-validate PyYAML

In [16]:
from datasets import load_dataset

ds = load_dataset("bigcode/the-stack", data_dir="data/yaml", split="train")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 1400000
})

In [19]:
import kubernetes_validate
import yaml

def validate(yaml_content: str):
    try:
        data = yaml.safe_load(yaml_content)
        data["Kind"] # will trigger KeyError and then return false if not valid
        kubernetes_validate.validate(data, '1.27', strict=True)
        return True
    except Exception as e:
        return False

validate(ds[0]["content"])

False

In [26]:
import os
os.cpu_count()

8

In [ ]:
%%time
valid_k8s = ds.filter(lambda batch: [validate(x) for x in batch["content"]], num_proc=os.cpu_count(), batched=True)

Filter (num_proc=8):   0%|          | 0/1400000 [00:00<?, ? examples/s]

In [37]:
valid_k8s

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 1669
})

In [38]:
valid_k8s.push_to_hub("substratusai/the-stack-yaml-k8s")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]